In [19]:
pip install networkx

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
pip install geopy


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
import pandas as pd
import requests
import folium
import networkx as nx
import random
import math
import numpy as np
from queue import PriorityQueue 
import heapq


df_preprocessing = pd.read_csv("airports-code@public.csv", sep=";")


df_preprocessing.head()

,Airport Code,Airport Name,City Name,Country Name,Country Code,Latitude,Longitude,World Area Code,City Name geo_name_id,Country Name geo_name_id,coordinates
0,CRS,Corsicana,Corsicana,United States,US,32.100000,-96.466667,67,4683462,6252001.0,"32.1, -96.466667"
1,ACP,Sahand,Sahand,Iran,IR,37.733333,46.450000,632,\N,130758.0,"37.733333, 46.45"
2,TBE,Timbunke,Timbunke,Papua New Guinea,PG,-4.183333,143.516667,804,\N,2088628.0,"-4.183333, 143.516667"
3,LWO,Snilow,Lviv,Ukraine,UA,49.812500,23.956111,488,702550,690791.0,"49.8125, 23.956111"
4,SCP,St Crepin,St Crepin,France,FR,44.683333,6.583333,427,\N,3017382.0,"44.683333, 6.583333"


In [22]:
# data preprossecing steps
df_preprocessing1 = df_preprocessing.drop(df_preprocessing.columns[[8,9]], axis = 1)
df_preprocessing1.head()

,Airport Code,Airport Name,City Name,Country Name,Country Code,Latitude,Longitude,World Area Code,coordinates
0,CRS,Corsicana,Corsicana,United States,US,32.100000,-96.466667,67,"32.1, -96.466667"
1,ACP,Sahand,Sahand,Iran,IR,37.733333,46.450000,632,"37.733333, 46.45"
2,TBE,Timbunke,Timbunke,Papua New Guinea,PG,-4.183333,143.516667,804,"-4.183333, 143.516667"
3,LWO,Snilow,Lviv,Ukraine,UA,49.812500,23.956111,488,"49.8125, 23.956111"
4,SCP,St Crepin,St Crepin,France,FR,44.683333,6.583333,427,"44.683333, 6.583333"


In [23]:
df_preprocessing[df_preprocessing.isnull().any(axis=1)]

,Airport Code,Airport Name,City Name,Country Name,Country Code,Latitude,Longitude,World Area Code,City Name geo_name_id,Country Name geo_name_id,coordinates
40,IDF,Idiofa,Idiofa,Zaire,ZR,-5.000000,19.508333,517,\N,NaN,"-5.0, 19.508333"
172,DGF,Bisbee-douglas International,Douglas,Unites States,US,31.468889,-109.653611,67,5823516,NaN,"31.468889, -109.653611"
289,LHR,London Heathrow,London,United Kingdom,UK,51.469722,-0.451389,493,\N,NaN,"51.469722, -0.451389"
302,INO,Inongo,Inongo,Democratic Republic of Congo,CD,-1.966667,18.266667,517,2315417,NaN,"-1.966667, 18.266666999999998"
307,USN,Ulsan,Ulsan,Reublic of Korea,KR,35.593611,129.352500,778,1833747,NaN,"35.593611, 129.3525"
...,...,...,...,...,...,...,...,...,...,...,...
8760,MSM,Masi Manimba,Masi Manimba,Zaire,ZR,-4.833333,17.850000,517,\N,NaN,"-4.833333, 17.85"
8906,HNO,Hercegnovi,Hercegnovi,Serbia and Montenegro,CS,42.566667,18.533333,497,\N,NaN,"42.566667, 18.533333"
8926,NKL,Nkolo,Nkolo,Zaire,ZR,-1.983333,16.466667,517,\N,NaN,"-1.983333, 16.466667"
9101,KGN,Kasongo Lunda,Kasongo Lunda,Zaire,ZR,-6.466667,16.816667,517,\N,NaN,"-6.466667, 16.816667"


In [24]:
def get_osrm_route(start_coords, end_coords):
    url = f"http://router.project-osrm.org/route/v1/driving/{start_coords[1]},{start_coords[0]};{end_coords[1]},{end_coords[0]}?overview=full&geometries=geojson"
    response = requests.get(url)
    
    if response.status_code == 200:
        route_data = response.json()
        distance_meters = route_data['routes'][0]['distance']  
        route_coords = route_data['routes'][0]['geometry']['coordinates']
        return route_coords, distance_meters
    else:
        return None, None


In [25]:
df_preprocessing1[['latitude', 'longitude']] = df_preprocessing1['coordinates'].str.split(',', expand=True)
df_preprocessing1['latitude'] = pd.to_numeric(df_preprocessing1['latitude'], errors='coerce')
df_preprocessing1['longitude'] = pd.to_numeric(df_preprocessing1['longitude'], errors='coerce')
df = df_preprocessing1.drop(df_preprocessing1.columns[[5,6]], axis = 1)
df.head()

,Airport Code,Airport Name,City Name,Country Name,Country Code,World Area Code,coordinates,latitude,longitude
0,CRS,Corsicana,Corsicana,United States,US,67,"32.1, -96.466667",32.100000,-96.466667
1,ACP,Sahand,Sahand,Iran,IR,632,"37.733333, 46.45",37.733333,46.450000
2,TBE,Timbunke,Timbunke,Papua New Guinea,PG,804,"-4.183333, 143.516667",-4.183333,143.516667
3,LWO,Snilow,Lviv,Ukraine,UA,488,"49.8125, 23.956111",49.812500,23.956111
4,SCP,St Crepin,St Crepin,France,FR,427,"44.683333, 6.583333",44.683333,6.583333


In [26]:

chosen_airport= ['Tokyo Haneda International','Soekarno-Hatta International', 'John F Kennedy Intl', 'Sydney', 'Beverley Springs', 'Ezeiza Ministro Pistarini', 'El Prat De Llobregat']
df_chosen_airports = df[df['Airport Name'].isin(chosen_airport)]
print(df_chosen_airports)

m = folium.Map(location=[df_chosen_airports['latitude'].mean(), df_chosen_airports['longitude'].mean()], zoom_start=3)

airport_locations = list(zip(df_chosen_airports['latitude'], df_chosen_airports['longitude']))
airport_names = df_chosen_airports['Airport Name'].tolist()

for index, row in df_chosen_airports.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']],
                  popup=f"{row['Airport Name']}<br>{row['City Name']}<br>{row['Country Name']}",).add_to(m)



     Airport Code                  Airport Name         City Name  \
100           BVZ              Beverley Springs  Beverley Springs   
2267          YQY                        Sydney        Sydney, NS   
2297          BCN          El Prat De Llobregat         Barcelona   
3443          CGK  Soekarno-Hatta International           Jakarta   
4000          JFK           John F Kennedy Intl          New York   
4662          HND    Tokyo Haneda International             Tokyo   
8520          EZE     Ezeiza Ministro Pistarini      Buenos Aires   

       Country Name Country Code  World Area Code             coordinates  \
100       Australia           AU              802      -17.983333, 125.45   
2267         Canada           CA              906   47.528056, -60.047222   
2297          Spain           ES              482          41.3, 2.083333   
3443      Indonesia           ID              832   -6.125556, 106.655833   
4000  United States           US               67   40.633333,

In [27]:
G = nx.Graph()

size = df_chosen_airports.shape[0]
for i in range(size):
    G.add_node(airport_names[i], pos=airport_locations[i])

from geopy.distance import geodesic

total_distance = 0
for i in range(len(airport_locations) - 1):
    for j in range(i + 1, len(airport_locations)): 
        start_coords = airport_locations[i]
        end_coords = airport_locations[j]
        
        # Compute geodesic distance in kilometers
        distance_km = geodesic(start_coords, end_coords).km
        total_distance += distance_km

        # Draw a straight line on the map
        route = [(start_coords[0], start_coords[1]), (end_coords[0], end_coords[1])]
        folium.PolyLine(locations=[route], color='blue', weight=2.5, opacity=0.8).add_to(m)
        
        # Add edge to the graph with distance as weight
        G.add_edge(airport_names[i], airport_names[j], weight=distance_km)

# Display the map
m


In [28]:


def addEdge(node, visited, pq, graph):
    visited[node] = True

    edges = graph.edges(node)
    for neighbor, edge_data in graph[node].items():
        if not visited[neighbor]:
            pq.put((edge_data['weight'], node, neighbor, graph.nodes[node]['pos'], graph.nodes[neighbor]['pos']))
        
    

def prims(graph, start):
    n = len(graph.nodes)
    visited = {node: False for node in graph.nodes} 
    m = graph.number_of_edges()
    edgeCount, mstCost = 0, 0
    mstEdges = []
    pq = PriorityQueue()

    addEdge(start, visited, pq, graph)


    while not pq.empty() and edgeCount < m:
        edge, startNode, endNode, startCoords, endCoords = pq.get()

        if visited[endNode] == True:
            continue
        
        visited[endNode] = True
        mstEdges.append((startNode, endNode, edge, startCoords, endCoords))
        edgeCount = edgeCount + 1
        mstCost = mstCost + edge

        addEdge(endNode, visited, pq, graph)

    return mstEdges, mstCost


In [29]:
mst_edges, mst_cost = prims(G, airport_names[0]) 

print(mst_edges)


m_mst = folium.Map(location=[df_chosen_airports['latitude'].mean(), df_chosen_airports['longitude'].mean()], zoom_start=3)

for index, row in df_chosen_airports.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']],
                  popup=f"{row['Airport Name']}<br>{row['City Name']}<br>{row['Country Name']}",).add_to(m_mst)


total_distance_mst = 0

count = 0
for edge in mst_edges:
    startNode, endNode, weight, startCoords, endCoords = edge

    route = [startCoords, endCoords]

    folium.PolyLine(locations=route, color='blue', weight=2.5, opacity=0.8).add_to(m_mst)
    
    total_distance_mst += weight
    count += 1

print(f"Total Distance of MST: {total_distance_mst} km")
print(count)

m_mst

[('Beverley Springs', 'Soekarno-Hatta International', 2426.84831266031, (-17.983333, 125.45), (-6.125556, 106.655833)), ('Soekarno-Hatta International', 'Tokyo Haneda International', 5770.022311061451, (-6.125556, 106.655833), (35.552222, 139.779722)), ('Tokyo Haneda International', 'El Prat De Llobregat', 10466.51647259595, (35.552222, 139.779722), (41.3, 2.083333)), ('El Prat De Llobregat', 'Sydney', 4863.156423943201, (41.3, 2.083333), (47.528056, -60.047222)), ('Sydney', 'John F Kennedy Intl', 1337.5399067983637, (47.528056, -60.047222), (40.633333, -73.783333)), ('John F Kennedy Intl', 'Ezeiza Ministro Pistarini', 8499.702778791238, (40.633333, -73.783333), (-34.822222, -58.535833))]
Total Distance of MST: 33363.78620585051 km
6


In [30]:
def preorderWalk(node, visited, adj_list, path):
    visited[node] = True
    path.append(node)

    for neighbor,_ in adj_list[node]:
        if not visited[neighbor]:
            preorderWalk(neighbor, visited, adj_list, path)


In [31]:

def construct_adjacency_list(mst_edges):
    adjacency_list = {}

    for startNode, endNode, edge, startCoords, endCoords in mst_edges:
        if startNode not in adjacency_list:
            adjacency_list[startNode] = []
        if endNode not in adjacency_list:
            adjacency_list[endNode] = []
        
        adjacency_list[startNode].append((endNode, edge))
        adjacency_list[endNode].append((startNode, edge))
    
    return adjacency_list

In [32]:

mst_adj_list = construct_adjacency_list(mst_edges)

visited = {node: False for node in mst_adj_list.keys()}

print(mst_adj_list['Sydney'][1][1])
path = []

preorderWalk(airport_names[0], visited, mst_adj_list, path) # perform a preOrder traversal
path.append(airport_names[0]) # create a eulerian cycle 

print(path)
print(mst_adj_list)


1337.5399067983637
['Beverley Springs', 'Soekarno-Hatta International', 'Tokyo Haneda International', 'El Prat De Llobregat', 'Sydney', 'John F Kennedy Intl', 'Ezeiza Ministro Pistarini', 'Beverley Springs']
{'Beverley Springs': [('Soekarno-Hatta International', 2426.84831266031)], 'Soekarno-Hatta International': [('Beverley Springs', 2426.84831266031), ('Tokyo Haneda International', 5770.022311061451)], 'Tokyo Haneda International': [('Soekarno-Hatta International', 5770.022311061451), ('El Prat De Llobregat', 10466.51647259595)], 'El Prat De Llobregat': [('Tokyo Haneda International', 10466.51647259595), ('Sydney', 4863.156423943201)], 'Sydney': [('El Prat De Llobregat', 4863.156423943201), ('John F Kennedy Intl', 1337.5399067983637)], 'John F Kennedy Intl': [('Sydney', 1337.5399067983637), ('Ezeiza Ministro Pistarini', 8499.702778791238)], 'Ezeiza Ministro Pistarini': [('John F Kennedy Intl', 8499.702778791238)]}


In [33]:
from math import radians, sin, cos, sqrt, atan2

In [34]:
print(path)

# Initialize the map centered around the first airport
m_path = folium.Map(location=[airport_locations[0][0], airport_locations[0][1]], zoom_start=6)

# Initialize total distance
total_distance = 0

# Iterate through the path and plot the points and lines
for i in range(len(path) - 1):
    start_name = path[i]
    end_name = path[i + 1]

    start_index = airport_names.index(start_name)
    end_index = airport_names.index(end_name)

    start_coords = airport_locations[start_index]
    end_coords = airport_locations[end_index]

    # Calculate the distance between start and end points using geodesic
    dist = geodesic(start_coords, end_coords).kilometers
    total_distance += dist

    # Draw the route on the map with a polyline
    route = [start_coords, end_coords]
    folium.PolyLine(locations=route, color='blue', weight=2.5, opacity=0.8).add_to(m_path)

    # Add marker for start point
    folium.Marker(location=start_coords, 
                  popup=f"{start_name}").add_to(m_path)

    # Add marker for end point (to avoid duplicate markers for consecutive points)
    if i == len(path) - 2:  # Add end marker only for the last iteration
        folium.Marker(location=end_coords, 
                      popup=f"{end_name}").add_to(m_path)

# Display the total distance
print(f"Total Distance: {total_distance:.2f} kilometers")

# Show map
m_path

['Beverley Springs', 'Soekarno-Hatta International', 'Tokyo Haneda International', 'El Prat De Llobregat', 'Sydney', 'John F Kennedy Intl', 'Ezeiza Ministro Pistarini', 'Beverley Springs']
Total Distance: 47508.66 kilometers


In [35]:
def convert_to_adjacency_matrix(adj_list, nodes):
    size = len(nodes)

    adj_matrix = np.zeros((size, size)) # init the matrix

    node_index = {}  
    for i, node in enumerate(nodes):  
        node_index[node] = i 

    for node in adj_list:
        row_index = node_index[node]
        for neighbor, weight in adj_list[node]:
            col_index = node_index[neighbor]
            adj_matrix[row_index][col_index] = weight
            adj_matrix[col_index][row_index] = weight
    
    return adj_matrix

In [36]:
networkx_adjacency_list = {}

for node in G.nodes:
    neighbor_list = []
    for neighbor in G.neighbors(node):
        weight = G[node][neighbor]['weight']
        neighbor_list.append((neighbor, weight))
    networkx_adjacency_list[node] = neighbor_list



adj_matrix = convert_to_adjacency_matrix(networkx_adjacency_list, airport_names)

print(adj_matrix)
print(networkx_adjacency_list)
print(G.number_of_edges())

[[    0.         16689.40653781 14083.27598754  2426.84831266
  16898.38183468  6115.07239525 14144.87470024]
 [16689.40653781     0.          4863.15642394 15245.08771197
   1337.5399068  10593.20591782  9122.0731163 ]
 [14083.27598754  4863.15642394     0.         11676.15396277
   6166.79972577 10466.5164726  10458.26827118]
 [ 2426.84831266 15245.08771197 11676.15396277     0.
  16181.18388172  5770.02231106 15215.35583847]
 [16898.38183468  1337.5399068   6166.79972577 16181.18388172
      0.         10899.62703627  8499.70277879]
 [ 6115.07239525 10593.20591782 10466.5164726   5770.02231106
  10899.62703627     0.         18358.42168228]
 [14144.87470024  9122.0731163  10458.26827118 15215.35583847
   8499.70277879 18358.42168228     0.        ]]
{'Beverley Springs': [('Sydney', 16689.406537812873), ('El Prat De Llobregat', 14083.275987536746), ('Soekarno-Hatta International', 2426.84831266031), ('John F Kennedy Intl', 16898.381834680797), ('Tokyo Haneda International', 6115.0723

In [37]:

def findZero(matrix):
    n = matrix.shape[1]
    coordinates = []
    for row in range(n):
        for col in range(n):
            if matrix[row, col] == 0:
                coordinates.append((row, col))

    return coordinates

def findPenalties(matrix, coordinates):
    penaltyDict = {}

    n = matrix.shape[1]

    for (row, col) in coordinates:
        
        if np.isinf(matrix[row, :]).all() or np.isinf(matrix[:, col]).all():
            continue
         # find penalty in row

        minRow = float('inf')
        for j in range(n):
            if j != col and not np.isinf(matrix[row, j]):
                minRow = min(minRow, matrix[row, j])

        minCol = float('inf')
        for i in range(n):
            if i != row and not np.isinf(matrix[i, col]):
                minCol = min(minCol, matrix[i, col])
        
        penaltyDict[(row, col)] = float(minRow + minCol)
    
    sortedPenalties = sorted(penaltyDict.items(), key=lambda x: x[1], reverse=True)
    sortedPenaltiesDict = dict(sortedPenalties)
    return sortedPenaltiesDict
         
def markMaxPenalties(penaltyDict, matrix, path):
    n = matrix.shape[1]
    coords = []

    for (row, col), penalty in penaltyDict.items():
        if matrix[row, col] == np.inf and (col, row) in coords:
            continue

        if (col, row) not in coords and (row, col) not in coords and penalty != 0:
            if matrix[row, col] != np.inf:
                coords.append((row, col))
                matrix[row, col] - np.inf
                matrix[row, :] = np.inf
                matrix[:, col] = np.inf

    return matrix, coords

def findPath(coords):
    adj_list = {}
    
    for x, y in coords:
        if x not in adj_list:
            adj_list[x] = []
        adj_list[x].append(y)

    all_paths = []

    visited = set()

    def dfs(node, path):
        path.append(node)
        visited.add(node)

        if node in adj_list:
            for neighbor in adj_list[node]:
                if neighbor not in visited:
                    dfs(neighbor, path)

        
    for startNode in adj_list.keys():
        if startNode not in visited:
            path = []
            dfs(startNode, path)
            all_paths.append(path)


    return all_paths


def findClosingLinks(path):
    copy = list(path)
    closingLinks = []
    for i in range(len(copy)):
        x = path[i][0]
        y = path[i][len(path[i]) - 1]
        closingLinks.append((x, y))

    return closingLinks

def changeMatrix(matrix, closingLinks, realMatrix):

    for i,_ in closingLinks:
        for _, j in closingLinks:
            matrix[j, i] = realMatrix[j, i]

    for i, j in closingLinks:
        matrix[j, i] = np.inf

    return matrix

def reduceMatrix(matrix):
    n = matrix.shape[0]

    for i in range(n):
        if not np.all(np.isinf(matrix[i, :])):
            minRowValue = np.min([matrix[i, j] for j in range(n) if not np.isinf(matrix[i, j])])
            for j in range(n):
                if not np.isinf(matrix[i, j]):
                    matrix[i, j] -= minRowValue

    for j in range(n):
        if not np.all(np.isinf(matrix[:, j])): 
            minColValue = np.min([matrix[i, j] for i in range(n) if not np.isinf(matrix[i, j])])
            for i in range(n):
                if not np.isinf(matrix[i, j]):
                    matrix[i, j] -= minColValue

    return matrix

def isComplete(matrix):
    n = matrix.shape[0]
    count = 0
    num = 0
    coords = []
    for i in range(n):
        for j in range(n):
            if matrix[i, j] != np.inf and matrix[i, j] != 0:
                count += 1
                coords.append([i, j])
    if count <= 1:
        return coords, True
    else:
        return -1,  False


def eulerianCycle(path, startingPoint):

    isComplete = False
    cycle = [i for i in path[0]]

    while not isComplete:
        for subpath in path:
            if subpath[0] == cycle[-1]:
                for x in subpath:
                    cycle.append(x)
        if len(cycle) > 1 and cycle[0] == cycle[-1]:
            isComplete = True
    
    i = 0
    while i < len(cycle) - 1:
        if cycle[i] == cycle[i + 1]:
            cycle.pop(i + 1)
        else:
            i += 1

    eulerianPath = []
    startIndex = 0
    for i in range(len(cycle)):
        if cycle[i] == startingPoint:
            eulerianPath.append(cycle[i])
            startIndex = i
            break
    
    index = startIndex + 1
    while len(eulerianPath) != len(cycle):
        if index == len(cycle) - 1:
            index = 0
        else:
            eulerianPath.append(cycle[index])
            index += 1
    
    
    return eulerianPath

def findTSPCost(eulerianPath):
    cost = 0

    for i in range(len(eulerianPath) - 1):
        cost += adj_matrix[eulerianPath[i]][eulerianPath[i + 1]]

    return cost

def hungarianAlgorithm(matrix, realMatrix):
    np.fill_diagonal(matrix, np.inf) 

    matrix = matrix - matrix.min(axis=1)[:, None]  
    matrix = matrix - matrix.min(axis=0) 

    complete = False

    n = matrix.shape[1]
    path = []  

    zeroCoordinates = findZero(matrix)
    penaltyDict = findPenalties(matrix, zeroCoordinates)

    matrix, coords = markMaxPenalties(penaltyDict, matrix, path)  
    path = findPath(coords)

    closingLinks = findClosingLinks(path)
    matrix = changeMatrix(matrix, closingLinks, realMatrix)

    while not complete:
        reduceMatrix(matrix)
        zeroCoordinates = findZero(matrix)

        penaltyDict = findPenalties(matrix, zeroCoordinates)
        matrix, coords = markMaxPenalties(penaltyDict, matrix, path) 

        secondPath = findPath(coords)
        path.extend(secondPath)

        num, complete = isComplete(matrix)
        path.extend(num)

    eulerianPath = eulerianCycle(path, 0)
    cost = findTSPCost(eulerianPath)

    return eulerianPath, cost


In [38]:
copyMatrix = adj_matrix
eulerianPath, cost = hungarianAlgorithm(copyMatrix, adj_matrix)
print(eulerianPath)
print(cost)

[0, 3, 5, 1, 4, 2, 6, 0]
50897.559145526044


In [42]:
def translatePath(eulerianPath):
    path = []
    for i in eulerianPath:
        path.append(airport_names[i])
    return path

hungarianPath = translatePath(eulerianPath)

print(hungarianPath)

['Beverley Springs', 'Soekarno-Hatta International', 'Tokyo Haneda International', 'Sydney', 'John F Kennedy Intl', 'El Prat De Llobregat', 'Ezeiza Ministro Pistarini', 'Beverley Springs']


In [43]:
m_Hungarian = folium.Map(location=[airport_locations[0][0], airport_locations[0][1]], zoom_start=6)
total_distance_hungarian = 0

for i in range(len(hungarianPath) - 1):
    start_name = hungarianPath[i]
    end_name = hungarianPath[i + 1]

    start_index = airport_names.index(start_name)
    end_index = airport_names.index(end_name)

    start_coords = airport_locations[start_index]
    end_coords = airport_locations[end_index]

    dist = geodesic(start_coords, end_coords).kilometers
    total_distance_hungarian += dist

    route = [start_coords, end_coords]

    folium.PolyLine(locations=route, color='blue', weight=2.5, opacity=0.8).add_to(m_Hungarian)

    folium.Marker(location=start_coords, popup=f"{start_name}").add_to(m_Hungarian)


    if i == len(path) - 2:  # Add end marker only for the last iteration
        folium.Marker(location=end_coords, popup=f"{end_name}").add_to(m_Hungarian)

print(f"Total Distance: {total_distance_hungarian:.2f} kilometers")
print(hungarianPath)
m_Hungarian

Total Distance: 50897.56 kilometers
['Beverley Springs', 'Soekarno-Hatta International', 'Tokyo Haneda International', 'Sydney', 'John F Kennedy Intl', 'El Prat De Llobregat', 'Ezeiza Ministro Pistarini', 'Beverley Springs']
